# Creating a SuperLearner Class and Fitting it on MNIST Fashion Dataset

### Imports

In [1]:
import random
from random import randrange
import pandas as pd
import numpy as np
from sklearn.tree import export_graphviz
from sklearn import metrics
from sklearn import tree
from sklearn import svm
from sklearn import ensemble
from sklearn import linear_model
from sklearn import neighbors
from sklearn import neural_network
from sklearn import naive_bayes
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from itertools import chain, combinations
from sklearn.base import BaseEstimator, ClassifierMixin
from IPython.display import display, HTML, Image
import numpy.random
from collections import Counter

### Functions

#### Function that reads in data:
Converts csv to dataframe and samples from according to sample rate.
Also normalises the X values by dividing by max pixel value (255) - note 0 corresponds to pure white and 255 to pure black

In [2]:
def Read_In_File(filename, sample_rate):
    """
    Reads in file by path and samples from it
    Divides all X values (after col 1) by 255 to normalise
    """
    dataset = pd.read_csv(filename)
    dataset = pd.DataFrame(data=dataset)
    dataset = dataset.sample(frac=sample_rate)
    dataset[dataset.columns[1:]] = dataset[dataset.columns[1:]] /255
    return(dataset)

#### Function that takes data and no. of folds and produces dict with those folds

In [3]:
def Divide_Into_Folds(data,num_folds,random_seed=5):
    """
    Takes data and number of folds and returns dict with each entry a fold 
    """
    for n in range(1,num_folds+1):
        locals()['fold'+str(n)] = []
    library_of_folds = dict()
    obs_each = data.shape[0]//num_folds # at least how many obs each fold
    extra = data.shape[0]%num_folds # spread these extra ones across folds
    random.seed(random_seed)
    randomlist = random.sample(range(len(data)),len(data))
    counter = 0
    for n in range(1,extra+1):
        locals()['fold'+str(n)] += [data.iloc[row] for row in randomlist[counter:counter+obs_each+1]]
        counter = counter+obs_each+1
    for n in range(extra+1,num_folds+1):
        locals()['fold'+str(n)] += [data.iloc[row] for row in randomlist[counter:counter+obs_each]]
        counter = counter+obs_each
    for n in range(1,num_folds+1):
        locals()['fold'+str(n)] = pd.DataFrame(data = locals()['fold'+str(n)])
        library_of_folds[n] = locals()['fold'+str(n)]
    return(library_of_folds)

#### Functions that returns a model based on some text entry (used for brevity)

In [4]:
def BaseModelName(name,criterion="entropy",min_samples_split = 200, \
                  min_samples_leaf = 50, max_depth=12, n_neighbors=20):
    """
    Returns model function for string entry specifying model
    Can also specify hyperparameters too
    """
    if name == 'dt':
        return(tree.DecisionTreeClassifier(criterion=criterion, \
                                           min_samples_split = min_samples_split, \
                                           min_samples_leaf = min_samples_leaf, \
                                           max_depth=max_depth))
    if name == 'lr':
        return(linear_model.LogisticRegression(multi_class='ovr', C=1, solver='liblinear', max_iter=1000))
    elif name == 'knn':
        return(neighbors.KNeighborsClassifier(n_neighbors=n_neighbors))
    elif name == 'mlp':
        return(neural_network.MLPClassifier(hidden_layer_sizes=(300, 100)))
    elif name == 'svm':
        return(svm.LinearSVC())
    elif name == 'nb':
        return(naive_bayes.GaussianNB())
    else:
        return("Error: Input Not A Correct Model Name")

### Load & Partition Data

Note that I read in a different dataset than the normal MNIST train dataset. I read in the first 7500 rows of that dataset, simply because I would get a Memory Error if I attempted to read in the whole dataset.

In [5]:
all_data = Read_In_File('fashion-mnist_train-cut.csv', 1)
all_data = all_data.reset_index(drop=True)
num_classes = 10
classes = {0: "T-shirt/top", 1:"Trouser", 2: "Pullover", 3:"Dress", 4:"Coat", 5:"Sandal", 6:"Shirt", 7:"Sneaker", 8:"Bag", 9:"Ankle boot"}

Now, expectedly the distribution of labels is not exactly distributed equally among the dataset so I ran through each row and deleted it if the count of the label of that row exceeded the minimum label count. 

In [6]:
for row in range(len(all_data)):
    counter = Counter(all_data["label"])
    label = all_data["label"][row]
    if counter[label] > min(counter.values()):
        all_data = all_data.drop(row)
        row -= 1

I then split the resulting dataset into X and Y datasets, and divided them further into training and test sets, both stratified by the label count in each.

In [7]:
X = all_data[all_data.columns[1:]]
Y = all_data[all_data.columns[0]]
labels = list(Counter(all_data["label"]).keys())
X_train, X_test, Y_train, Y_test \
= train_test_split(X, Y, random_state=89, train_size = 0.8, stratify = Y)

C:\Users\Luke\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Function that returns a bootstrapped sample. So sample with replacement, and size of sample determined by inputted fraction of size of dataset

In [8]:
def Bootstrap(data, size_as_fraction):
    """
    Takes in data (assumes data frame) and returns sample of data 
    (with size of sample specified by size_as_fraction and sample with replacement)
    """
    n = len(data)
    index = numpy.random.randint(0, n, n)
    sample = []
    sample += [data.iloc[row] for row in index]
    sample = pd.DataFrame(sample)
    sample = sample.sample(frac=size_as_fraction)
    return (sample)

### Super Learner Class
This class defined by the three functions: **initialises** class variables, by assigning their values by input commands or by default commands, and adding some self variables that are simply pawns in helping the whole funcitonality of the class; **fitting** model, by taking an X and Y dataset, combining, training base estimators on cross validated datasets of these and outputting predictions which are to be used in training the stack estimator on the resulting so-called Z dataset. The base estimators are then trained on bootstrapped samples of the whole training set. Finally **predicting** the response values of a test set by using predictions by base estimators and a stack model to predict the responses using these predictions. Note I constructed the predict function in similar form to the fit function, but frankly, there is absolutely no need for folds in the predict function, as there are no responses to fit to, so it would be more efficient to simply predict the entire test set.

In [9]:
class SuperLearnerClassifier(BaseEstimator, ClassifierMixin):  
    """
    SuperLearnerClassifier Class
    """
    
    def __init__(self, base_model_list = ['dt','dt','dt'], \
                 stack_model = 'dt', type_output_labels = 'predictions', \
                 add_orgdata = 'yes', bootstrap_size = 0.8, rand_seed = 3):
        self.base_model_list=base_model_list
        self.stack_model=stack_model
        self.type_output_labels=type_output_labels
        self.add_orgdata=add_orgdata
        self.bootstrap_size = bootstrap_size
        self.rand_seed=rand_seed
        self.num_base_estimators = len(self.base_model_list)
        # Define stack model and base models (up to 10) to be used later
        self.Zdata = None
        self.ult_model = None
        self.model_list = []
        for i in range(1,11):
            self.model_list.append('model'+str(i))
        for model in self.model_list:
            setattr(self, model, None)

    def fit(self,X_train_data,Y_train_data):        
        X_train_data = pd.DataFrame(data=X_train_data)
        Y_train_data = pd.DataFrame(data=Y_train_data)
        X_train_data = X_train_data.reset_index(drop=True)
        Y_train_data = Y_train_data.reset_index(drop=True)
        train_data = pd.merge(Y_train_data, X_train_data, left_index=True, right_index=True)
        # so reset X and Y and merge to one so response values
        # are in correct order so easy to train
        folds_dict = Divide_Into_Folds(train_data,self.num_base_estimators,self.rand_seed)
        # returns dict of folds, no of folds determined by no. of base estimators
        for k in range(self.num_base_estimators):
            other_keys = [m for m in folds_dict.keys() if m!=k+1]
            appended_list = [folds_dict[m] for m in other_keys[1:]]
            train_set = folds_dict[other_keys[0]].append(appended_list,ignore_index=True)
            test_set = folds_dict[k+1]
            # train set is amalgamated all folds other than k
            # test set is kth fold
            for n,each_model in enumerate(self.base_model_list):
                model = BaseModelName(each_model)
                # initialise new model according to string in list
                model = model.fit(train_set[train_set.columns[1:]],train_set[train_set.columns[0]])
                if self.type_output_labels == 'probabilities':
                    y_pred = model.predict_proba(test_set[test_set.columns[1:]])
                else:
                    y_pred = model.predict(test_set[test_set.columns[1:]])
                # if output labels are probabilities use predict_proba otherwise predict
                y_pred = pd.DataFrame(data=y_pred)
                if n+1 == 1:
                    temp_pred = y_pred
                else:
                    temp_pred = pd.merge(temp_pred, y_pred, left_index=True, right_index=True)
                # for each fold, merge predictions from each base estimator
                # if 1st output labels then initialise, otherwise add to existing list
            if k+1 == 1:
                all_pred = temp_pred
            else:
                all_pred = all_pred.append(temp_pred,ignore_index=True)
            # for each fold, merge predictions from last fold, to get set of predictions
            # for all rows
        full_data = folds_dict[1].append([folds_dict[m] for m in range(2,self.num_base_estimators+1)],ignore_index=False)
        # combine folds to new dataset that is in order of predictions above
        if self.add_orgdata == 'no':
            responses = pd.DataFrame(full_data[full_data.columns[0]])
            new_train = pd.merge(responses, all_pred, left_index=True, right_index=True)
        else:
            new_train = pd.merge(full_data, all_pred, left_index=True, right_index=True)
        # if not using original data in dataset Z to accompany output labels of base estimators
        # then merge with X data to get the right order then exclude those cols
        # if use original data too then merge with X data
        new_train.sort_index(inplace=True)
        new_train.reset_index(drop=True)
        # sort training set so outputted predictions are in same order as input data
        self.ult_model = BaseModelName(self.stack_model)
        self.ult_model = self.ult_model.fit(new_train[new_train.columns[1:]],new_train[new_train.columns[0]])
        # initialise and set stack model
        for m,each_model in enumerate(self.base_model_list):
            setattr(self, 'model'+str(m+1), BaseModelName(each_model))
            bootstrap_sample = Bootstrap(train_data,self.bootstrap_size)
            setattr(self, 'model'+str(m+1), getattr(self, 'model'+str(m+1)).\
                        fit(bootstrap_sample[bootstrap_sample.columns[1:]],bootstrap_sample[bootstrap_sample.columns[0]]))
        # train each of the base estimators on bootstrapped samples of original data
        # to be used in predicting test set
        return(self)

    def predict(self,X_test_data):
        # organised in very similar fashion to fit function so comments there
        # hopefully satisfice to explain the thoughts behind the code
        X_test_data = pd.DataFrame(data=X_test_data)
        X_test_data = X_test_data.reset_index(drop=True)
        folds_dict = Divide_Into_Folds(X_test_data,self.num_base_estimators,self.rand_seed)
        for k in range(self.num_base_estimators):
            test_set = folds_dict[k+1]
            for n,model in enumerate(self.base_model_list):
                if self.type_output_labels == 'probabilities':
                    y_pred = getattr(self, 'model'+str(n+1)).predict_proba(test_set)
                else:
                    y_pred = getattr(self, 'model'+str(n+1)).predict(test_set)
                y_pred = pd.DataFrame(data=y_pred)
                if n+1 == 1:
                    temp_pred = y_pred
                else:
                    temp_pred = pd.merge(temp_pred, y_pred, left_index=True, right_index=True)                
            if k+1 == 1:
                all_pred = temp_pred
            else:
                all_pred = all_pred.append(temp_pred,ignore_index=True)
        full_data = folds_dict[1].append([folds_dict[m] for m in range(2,self.num_base_estimators+1)],ignore_index=False)
        self.Zdata = pd.merge(full_data,all_pred, how='inner', left_index=True,right_index=True )
        self.Zdata.sort_index(inplace=True)
        self.Zdata.reset_index(drop=True)
        if self.add_orgdata == 'no':
            col = full_data.shape[1]
            self.Zdata = self.Zdata[self.Zdata.columns[col:]]
        predictions = self.ult_model.predict(self.Zdata)
        return(predictions)

In [10]:
sl = SuperLearnerClassifier(base_model_list = ['dt','lr','nb'], \
                            stack_model = 'dt',type_output_labels = 'predictions',\
                            add_orgdata = 'no', bootstrap_size = 0.8, rand_seed = 30)

In [11]:
scores = cross_val_score(sl, X_train, Y_train, cv=3)

In [12]:
print(sum(scores)/len(scores))

0.09702920038251743


### The following code takes a long time to run

Compare score to basic models

In [ ]:
dec_tree = BaseModelName('dt')
scores_dt = cross_val_score(dec_tree, X_train, Y_train, cv=3)
print(sum(scores_dt)/len(scores_dt))  
nav_bayes = BaseModelName('nb')
scores_nb = cross_val_score(nav_bayes, X_train, Y_train, cv=3)
print(sum(scores_nb)/len(scores_nb))  
knn = BaseModelName('knn')
scores_knn = cross_val_score(knn, X_train, Y_train, cv=3)
print(sum(scores_knn)/len(scores_knn)) 
mlp = BaseModelName('mlp')
scores_mlp = cross_val_score(mlp, X_train, Y_train, cv=3)
print(sum(scores_mlp)/len(scores_mlp))

Find best combination of parameters

In [ ]:
def Powerset(iterable):
    """
    Returns powerset by with restraint that no. of elements >= 2
    so Powerset([1,2,3]) --> (1,2) (1,3) (2,3) (1,2,3)
    """
    constituents = list(iterable)
    return chain.from_iterable(combinations(constituents,n) \
                               for n in range(2,len(constituents)+1))

all_possible_base_model_combos = list(Powerset(['dt','lr','knn','mlp','nb'])) # svm omitted due to no predict_proba() function

Note I omitted svm as it did not have the predict_proba() function for some reason.I did not run the grid search through all possible combinations of base estimator alliances as that would take an inordinate length of time to complete (342 combinations of >2 different base estimators when combined with iterating over all the values of stack estimators and output labels). So I randomly sampled from each number of base estimators (so for example 2 sampled combinations of base estimators could be ['dt','knn'] and ['mlp',nb']); 2 combinations of 2 base estimators, 2 of 3, 2 of 4, and 1 of 5. The higher combinations having less samples as there are less of them. The grid search also iterated over stack estimators (7), and outputs to train the stack model on (2). 

In [ ]:
all_combinations = [list(combo) for combo in all_possible_base_model_combos]
models2, models3, models4, models5 = [], [], [], []
for i in range(2,6):
    for combo in all_combinations:
        if len(combo)==i:
            locals()["models"+str(i)] += [combo]

models2 = [models2[j] for j in sorted(random.sample(range(len(models2)), 2))]
models3 = [models3[j] for j in sorted(random.sample(range(len(models3)), 2))]
models4 = [models4[j] for j in sorted(random.sample(range(len(models4)), 2))]
# models5 only contains one element so no need to sample from
        
some_combinations = models2+models3+models4+models5
param_grid ={'base_model_list': [model_list for model_list in some_combinations], \
             'stack_model': ['dt','knn','nb','lr','svm','mlp'], \
             'type_output_labels': ['predictions','probabilities'],\
             'bootstrap_size' : [0.65],
             'add_orgdata': ['no'], 'rand_seed': [3]}
grid_search = GridSearchCV(SuperLearnerClassifier(), param_grid, cv=2, \
                             verbose = 1, return_train_score=True)
grid_search.fit(X_train, Y_train)
print("Best parameters set found on development set:")
display(grid_search.best_params_)
display(grid_search.best_score_)
display(grid_search.cv_results_)


Compare performance when training on the original data values or not

In [ ]:
sl_withorgdata = SuperLearnerClassifier(base_model_list = ['dt','lr','nb'], \
                            stack_model = 'nb',type_output_labels = 'predictions',\
                            add_orgdata = 'yes', bootstrap_size = 0.65, rand_seed = 30)
sl_withorgdata = sl_withorgdata.fit(X_train, Y_train)
y_sl_withorgdata = sl_withorgdata.predict(X_test)
print(metrics.accuracy_score(Y_test, y_sl_withorgdata))
sl_withoutorgdata = SuperLearnerClassifier(base_model_list = ['dt','lr','nb'], \
                            stack_model = 'nb',type_output_labels = 'predictions',\
                            add_orgdata = 'no', bootstrap_size = 0.65, rand_seed = 30)
sl_withoutorgdata = sl_withoutorgdata.fit(X_train, Y_train)
y_sl_withoutorgdata = sl_withoutorgdata.predict(X_test)
print(metrics.accuracy_score(Y_test, y_sl_withoutorgdata))